# Spatial queries
By now, you should have some idea of how to explore the various ``voeventdb.remote`` endpoints and apply filters. 

In this notebook, we'll extend our filter-toolset to include cone-searches.


In [ ]:
from __future__ import print_function
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
import voeventdb.remote as vr
import voeventdb.remote.apiv1 as apiv1
from voeventdb.remote.apiv1 import FilterKeys
from voeventdb.remote.helpers import Synopsis
from astropy.coordinates import Angle, SkyCoord

In the last tutorial, we inspected a  *Swift* XRT event which gave us a position estimate and and error-circle. Let's grab those details again:

In [ ]:
xrt_synopsis = Synopsis(apiv1.synopsis('ivo://nasa.gsfc.gcn/SWIFT#XRT_Pos_666352-553'))

In [ ]:
sky_event = xrt_synopsis.sky_events[0]
print(sky_event)
print("Error-circle radius in degrees, arcminutes and arcseconds:", sky_event.position_error)

## Filtering using a cone-search (AKA spatial queries)
Next, let's see if there's any other recorded events with associated positions nearby. To do so, we'll need to define a 'cone', a sky-position and circle around it to search in. For setting up a ``voeventdb.remote`` cone-filter, we can use a tuple of type 

``(astropy.coordinates.SkyCoord, astropy.coordinates.Angle)``

A natural candidate is the position and position-error from the XRT event; like so:

In [ ]:
cone = (sky_event.position, sky_event.position_error)
cone

However, the XRT position has a really tight error-circle, about 5 arcseconds. Note that the cone-search will only return VOEvents with a *best-estimate position* within the cone - **it does not take into account overlapping error-circles** (at least for version 1!). This means that we could have a related event with a large error-circle just outside the tiny XRT error-circle, and it wouldn't be returned - so we have to use some judgement here. We'll set the cone angular radius to half a degree, instead:

In [ ]:
cone = (sky_event.position, Angle(0.5, unit='deg'))
cone

OK, let's see how that works:

In [ ]:
cone_filters = {
    FilterKeys.role: 'observation',
    FilterKeys.cone: cone
    }

In [ ]:
apiv1.stream_count(cone_filters)

A reasonable number. Let's take a look:

In [ ]:
sorted(apiv1.ivorn(cone_filters))

So, we have a bunch of packets related to the same event (ID 666352), some 'SubSubThresh' events, and a 'Known_Pos' event. It's worth noting that 'SubSubThresh' events are extremely common and show up all over:

In [ ]:
all_count = apiv1.count(filters={FilterKeys.role:'observation'})
ss_thresh_count = apiv1.count(filters={
        FilterKeys.role:'observation', 
        FilterKeys.ivorn_contains:'BAT_SubSubThresh',
        })
print("Of {} observation packets in the database, {} are BAT_SubSubThresh packets".format(
        all_count, ss_thresh_count))

So it's perhaps not surprising that we'd encounter a few of them co-incidentally lying in the search-cone.
We can define a search-cone at arbitrary co-ordinates and see what we get back, for comparison:

In [ ]:
cone2 = (SkyCoord(ra=0, dec=35., unit='deg'), Angle(0.5, unit='deg'))
cone2

In [ ]:
cone2_filters = {
    FilterKeys.role: 'observation',
    FilterKeys.cone: cone2
    }

In [ ]:
apiv1.stream_count(filters=cone2_filters)

In [ ]:
apiv1.ivorn(filters=cone2_filters)

Result - looks like we get a similar number of these events where-ever we point our search-cone!

## Coming next ...
In the search above for packets near to a Swift XRT position, many of the packets were clearly designated as relating to the same event - they all had the same ID number. Is that relation encoded into the packet-data? Can we just select the VOEvents which are already marked as being related to a particular packet?

The answer, of course, is yes - see the next notebook.